In [ ]:
import cv2
#画像ファイルを読み込む
image = cv2.imread("path")

# クリックイベントのコールバック関数
def click_event(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:  # 左クリックが行われた場合
        print(f'クリックされた座標: ({x}, {y})')

# 画像を大きく表示
cv2.namedWindow("Image", cv2.WINDOW_NORMAL)
cv2.imshow("Image", image)

# クリックイベントを待機
cv2.setMouseCallback("Image", click_event)
cv2.waitKey(0)

# ウィンドウを閉じる
cv2.destroyAllWindows()


In [ ]:
import cv2
import matplotlib.pyplot as plt

# 画像ファイルを読み込む
img = cv2.imread("path")

＃任意の比率を入力
img1 = img[　:　, 　:　]

plt.imshow(img1)
plt.show()

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize, curve_fit

# グレースケール形式に変換
gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)

# 平滑化フィルタ
smooth_img = cv2.blur(gray, (5, 5))

# Cannyのエッジ検出
canny_img = cv2.Canny(smooth_img, 1, 140)

# 輪郭を見つける
contours, _ = cv2.findContours(canny_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

# 白い曲線の点の座標を収集するリスト
points = []

for contour in contours:
    for point in contour:
        x, y = point[0]
        points.append((x, y))

# 座標情報を抽出
x_coordinates, y_coordinates = zip(*points)

# タプルからリストに変換
x_coordinates = list(x_coordinates)
y_coordinates = list(y_coordinates)


# 画像サイズを取得
image_height, image_width = img1.shape[:2]


# 新しい範囲を計算
x_min = min(x_coordinates)
x_max = max(x_coordinates)
new_x_start = x_min + (x_max - x_min) / 3
new_x_end = x_min + 2 * (x_max - x_min) / 3

# 指定した範囲内の点をx座標でソートし、小さい順に表示
points_in_range = [(x, y) for x, y in zip(x_coordinates, y_coordinates) if x >= new_x_start and x <= new_x_end]
sorted_points = sorted(points_in_range, key=lambda point: point[0])

# クラスタリング用の中点を計算
middle_points = []

for x in range(int(new_x_start), int(new_x_end) + 1):
    x_values = [point[1] for point in sorted_points if point[0] == x]
    if len(x_values) == 2:
        y_middle = sum(x_values) / 2
        middle_points.append((x, y_middle))
    elif len(x_values) >= 3:
    # y_coordinatesからx_valuesに対応するy座標のリストを作成
        y_values = [y_coordinates[i] for i, x_val in enumerate(x_coordinates) if x_val == x]
    # y_valuesをソート
        y_values.sort()
    
    # 隣り合う点同士の距離を計算
    distances = [abs(y_values[i] - y_values[i + 1]) for i in range(len(y_values) - 1)]
    
    # 最大距離を持つ隣り合う点のy座標を取得
    max_distance_index = distances.index(max(distances))
    max_point1 = y_values[max_distance_index]
    max_point2 = y_values[max_distance_index + 1]
    
    # 中点を計算してmiddle_pointsに追加
    y_middle = (max_point1 + max_point2) / 2
    middle_points.append((x, y_middle))

middle_x, middle_y = zip(*middle_points)

# 対数関数の定義
def logarithmic_model(x, a, b, c):
    return a * np.log(b * x) + c

# 最小二乗法を使用して対数近似曲線を計算
params, _ = curve_fit(logarithmic_model, middle_x, middle_y)

# クラスタリング
front_wall_points = []
back_wall_points = []

for x, y in points:
    if y > logarithmic_model(x, *params):
        back_wall_points.append((x, y))
    else:
        front_wall_points.append((x, y))

front_wall_points = np.array(front_wall_points)
back_wall_points = np.array(back_wall_points)

# クラスタ2のx座標が最小と最大の点を求めて中点を計算
min_x = np.min(back_wall_points[:, 0])
max_x = np.max(back_wall_points[:, 0])
center2_x_values = [(min_x + max_x) / 2 - 3,(min_x + max_x) / 2 - 2,(min_x + max_x) / 2 - 1,(min_x + max_x) / 2, (min_x + max_x) / 2 + 1, (min_x + max_x) / 2 + 2, (min_x + max_x) / 2 + 3]

# リストを初期化
line_lengths = []

# center2_x を変えながら直線の長さを計算
for center2_x in center2_x_values:
    # クラスタ2の中で基準点のx座標と等しいx座標の点を求める
    x_cluster2 = center2_x  # クラスタ2の基準点のx座標

    # クラスタ2内からx座標が一致する点を抽出
    matching_points = [point for point in back_wall_points if point[0] == x_cluster2]

    if len(matching_points) == 1:
        # x座標が一致する点が1つだけ存在する場合、それを選択
        closest_point_cluster2 = matching_points[0]
    else:
        # x座標が一致する点が複数存在する場合、y座標が最大の点を選択
        max_y = min(matching_points, key=lambda point: point[1])
        closest_point_cluster2 = max_y

    # クラスタ1の中で基準点に最も近い点を求める
    closest_point_cluster1 = front_wall_points[np.argmin(np.linalg.norm(front_wall_points - closest_point_cluster2, axis=1))]

    # 基準点と最も近い点を結ぶ直線を計算
    line_length = np.linalg.norm(closest_point_cluster2 - closest_point_cluster1)

    line_lengths.append((center2_x, line_length))

# 1ピクセルあたりのサイズ（mm）
pixel_size_mm = 

# 直線の長さをピクセルからmmに変換して表示
for center2_x, length in line_lengths:
    length_mm = length * pixel_size_mm
    print(f"Center2_x: {center2_x}, Line Length (mm): {length_mm:.2f} mm")

# 直線の長さの平均値を計算
max_average_length_mm = np.mean([length[1] * pixel_size_mm for length in line_lengths])
print(f"max_Average Line Length (mm): {max_average_length_mm:.2f} mm")

# グラフに特徴点をプロット
plt.scatter(front_wall_points[:, 0], image_height - front_wall_points[:, 1], c='green', marker='o', s=1, label='Front Wall')  # 青い点でプロット
plt.scatter(back_wall_points[:, 0], image_height - back_wall_points[:, 1], c='blue', marker='o', s=1, label='Back Wall')  # 緑の点でプロット

# 直線を描画
plt.plot([closest_point_cluster2[0], closest_point_cluster1[0]], [image_height - closest_point_cluster2[1], image_height - closest_point_cluster1[1]], c='purple', linestyle='--', linewidth=2, label='Line')

# グラフのタイトルと軸ラベルを設定
plt.title('Feature Points Clusters and Line')
plt.xlabel('X')
plt.ylabel('Y')

# 軸の最大値・最小値の設定
plt.xlim(0, image_width)
plt.ylim(0, image_height)

plt.axes().set_aspect('equal')

# クラスタリングの結果を表示
plt.legend()

# 表示
plt.grid()
plt.show()


In [ ]:
import cv2
import numpy as np

# 呼吸性変動率の計算
respiratory_variation = (max_average_length_mm - min_average_length_mm) /max_average_length_mm * 100 

# 呼吸性変動率の結果を表示
print(f"呼吸性変動率(%):, {respiratory_variation:.2f} %")